In [400]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
from IPython.display import display



NA_VALUES = ["---", "--", "", " ", "NA", "N/A"]
# =========================
# FILE FINDER
# =========================
def find_file(name, start=Path.cwd()):
    for ancestor in [start] + list(start.parents):
        matches = list(ancestor.rglob(name))
        if matches:
            return matches[0]
    return None


def find_files(file_map):
    found = {}
    for key, filename in file_map.items():
        path = find_file(filename)
        if path:
            found[key] = path
        else:
            print(f"[WARNING] File not found: {filename}")
    return found

eda_script_path = find_file("script_eda.py")
if eda_script_path is None:
    raise FileNotFoundError("❌ script_eda.py tidak ditemukan di parent directory")

# tambahkan BASE PROJECT ke sys.path
sys.path.append(str(eda_script_path.parent))

# sekarang bisa import
from script_eda import evaluate_dataset, extract_column_schema,find_internal_duplicate_columns,extract_single_schema,cek_value_data_column


In [401]:
path = find_file("ispu_all_years.csv")

if path is None:
    raise FileNotFoundError("❌ File merged tidak ditemukan")

df = pd.read_csv(path, na_values=NA_VALUES)
df.head()

,periode_data,tanggal,stasiun,pm_sepuluh,pm_duakomalima,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,max,parameter_pencemar_kritis,kategori,id
0,201001,2010-01-19,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-19_DKI2
1,201001,2010-01-20,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-20_DKI2
2,201001,2010-01-21,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-21_DKI2
3,201001,2010-01-22,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-22_DKI2
4,201001,2010-01-23,DKI2 (Kelapa Gading),NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,TIDAK ADA DATA,2010-01-23_DKI2


In [402]:
df["stasiun_clean"] = (
    df["stasiun"]
    .astype(str)
    .str.extract(r"(DKI\d)")
)
df["stasiun_clean"].value_counts()



stasiun_clean
DKI4    3717
DKI2    3491
DKI5    3408
DKI3    3355
DKI1    2898
Name: count, dtype: int64

In [403]:
df["tanggal_raw"] = df["tanggal"].astype(str).str.strip()

# kosong / tanda minus → NaN
df.loc[df["tanggal_raw"].isin(["", "-", "nan", "None"]), "tanggal_raw"] = pd.NA

# siapkan kolom hasil parsing
df["tanggal"] = pd.NaT

In [404]:
mask_iso = df["tanggal_raw"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)

df.loc[mask_iso, "tanggal"] = pd.to_datetime(
    df.loc[mask_iso, "tanggal_raw"],
    format="%Y-%m-%d",
    errors="coerce"
)


In [405]:
mask_us = df["tanggal_raw"].str.match(r"^\d{1,2}/\d{1,2}/\d{4}$", na=False)

df.loc[mask_us, "tanggal"] = pd.to_datetime(
    df.loc[mask_us, "tanggal_raw"],
    format="%m/%d/%Y",
    errors="coerce"
)


In [406]:
mask_remaining = df["tanggal"].isna()

df.loc[mask_remaining, "tanggal"] = pd.to_datetime(
    df.loc[mask_remaining, "tanggal_raw"],
    errors="coerce"
)


In [407]:
print("Jumlah NaT:", df["tanggal"].isna().sum())
df[df["tanggal"].isna()][["tanggal_raw"]].head(20)


Jumlah NaT: 106


,tanggal_raw
2851,<NA>
2852,<NA>
2853,<NA>
2854,<NA>
2855,<NA>
2856,<NA>
2857,<NA>
2858,<NA>
2859,<NA>
2860,<NA>


In [408]:
df[df["tanggal"].isna()].groupby("stasiun_clean").size().sort_values(ascending=False)


stasiun_clean
DKI3    29
DKI5    29
DKI4    28
DKI2    20
dtype: int64

In [409]:
df_nat = df[df["tanggal"].isna()].copy()
df_nat.groupby(["stasiun_clean", "periode_data"]).size().sort_values(ascending=False)


stasiun_clean  periode_data
DKI2           201612          19
DKI3           201612          19
DKI4           201612          19
DKI5           201612          19
DKI3           201309          10
DKI5           201309          10
DKI4           201309           9
DKI2           201309           1
dtype: int64

In [410]:
df = df.dropna(subset=["tanggal"]).copy()

print("Sisa NaT:", df["tanggal"].isna().sum())
print("Shape setelah drop:", df.shape)


Sisa NaT: 0
Shape setelah drop: (16796, 15)


In [411]:
df = df.sort_values(["stasiun_clean", "tanggal"]).reset_index(drop=True)


In [412]:
df["delta_hari"] = (
    df.groupby("stasiun_clean")["tanggal"]
    .diff()
    .dt.days
)


In [413]:
df["stasiun_clean"] = (
    df["stasiun"]
    .astype(str)
    .str.extract(r"(DKI\d)")
)
df["stasiun_clean"].value_counts()



stasiun_clean
DKI4    3689
DKI2    3471
DKI5    3379
DKI3    3326
DKI1    2898
Name: count, dtype: int64

In [414]:
df["tanggal_raw"] = df["tanggal"].astype(str).str.strip()

# kosong / tanda minus → NaN
df.loc[df["tanggal_raw"].isin(["", "-", "nan", "None"]), "tanggal_raw"] = pd.NA

# siapkan kolom hasil parsing
df["tanggal"] = pd.NaT

In [415]:
mask_iso = df["tanggal_raw"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)

df.loc[mask_iso, "tanggal"] = pd.to_datetime(
    df.loc[mask_iso, "tanggal_raw"],
    format="%Y-%m-%d",
    errors="coerce"
)


In [416]:
mask_us = df["tanggal_raw"].str.match(r"^\d{1,2}/\d{1,2}/\d{4}$", na=False)

df.loc[mask_us, "tanggal"] = pd.to_datetime(
    df.loc[mask_us, "tanggal_raw"],
    format="%m/%d/%Y",
    errors="coerce"
)


In [417]:
mask_remaining = df["tanggal"].isna()

df.loc[mask_remaining, "tanggal"] = pd.to_datetime(
    df.loc[mask_remaining, "tanggal_raw"],
    errors="coerce"
)


In [418]:
print("Jumlah NaT:", df["tanggal"].isna().sum())
df[df["tanggal"].isna()][["tanggal_raw"]].head(20)


Jumlah NaT: 0


,tanggal_raw


In [419]:
df[df["tanggal"].isna()].groupby("stasiun_clean").size().sort_values(ascending=False)


Series([], dtype: int64)

In [420]:
df_nat = df[df["tanggal"].isna()].copy()
df_nat.groupby(["stasiun_clean", "periode_data"]).size().sort_values(ascending=False)


Series([], dtype: int64)

In [421]:
df = df.dropna(subset=["tanggal"]).copy()

print("Sisa NaT:", df["tanggal"].isna().sum())
print("Shape setelah drop:", df.shape)


Sisa NaT: 0
Shape setelah drop: (16796, 16)


In [422]:
df["delta_hari"] = (
    df.groupby("stasiun_clean")["tanggal"]
    .diff()
    .dt.days
)


In [423]:
df["delta_hari"].describe()


count    16758.000000
mean         1.706946
std          8.087351
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        705.000000
Name: delta_hari, dtype: float64

In [424]:
df["parameter_pencemar_kritis"].value_counts()


parameter_pencemar_kritis
PM25     5574
O3       5411
PM10     3027
SO2       494
PM2,5     333
CO        247
2         112
4          31
1          30
NO2        22
-           8
3           7
5           3
BAIK        1
Name: count, dtype: int64

In [425]:
mapping = {
    "1": "PM10",
    "2": "PM25",
    "3": "SO2",
    "4": "CO",
    "5": "O3",
    "PM2,5": "PM25",
    "BAIK": "PM25",
    "-": pd.NA
}

df["parameter_pencemar_kritis"] = (
    df["parameter_pencemar_kritis"]
    .astype(str)
    .str.strip()
    .replace(mapping)
)
df["parameter_pencemar_kritis"].value_counts()

parameter_pencemar_kritis
PM25    6020
O3      5414
PM10    3057
nan     1496
SO2      501
CO       278
NO2       22
Name: count, dtype: int64

In [426]:
df.isna().sum()


periode_data                    0
tanggal                         0
stasiun                         0
pm_sepuluh                   1937
pm_duakomalima               9889
sulfur_dioksida              1724
karbon_monoksida             1623
ozon                         1774
nitrogen_dioksida            1702
max                            12
parameter_pencemar_kritis       8
kategori                        1
id                              0
stasiun_clean                  33
tanggal_raw                     0
delta_hari                     38
dtype: int64

In [427]:
df = df[df["kategori"] != "TIDAK ADA DATA"].copy()
mapping_kategori = {
    "SANGAT TIDAK SEHAT": "TIDAK SEHAT",
    "BERBAHAYA": "TIDAK SEHAT"
}

df["kategori"] = df["kategori"].replace(mapping_kategori)
df["kategori"].value_counts()


kategori
SEDANG         10419
TIDAK SEHAT     2608
BAIK            2279
O3                31
Name: count, dtype: int64

In [428]:
df_short = df[
    (df["tanggal"].dt.year >= 2021) &
    (df["tanggal"].dt.year <= 2025)
].copy()


In [429]:
df_short["kategori"].value_counts()

kategori
SEDANG         5241
TIDAK SEHAT     930
BAIK            805
Name: count, dtype: int64

In [430]:
df_short.shape


(6977, 16)

In [431]:
df_short.isna().sum()


periode_data                   0
tanggal                        0
stasiun                        0
pm_sepuluh                   220
pm_duakomalima               119
sulfur_dioksida              133
karbon_monoksida              47
ozon                          78
nitrogen_dioksida             67
max                            0
parameter_pencemar_kritis      0
kategori                       1
id                             0
stasiun_clean                  1
tanggal_raw                    0
delta_hari                     1
dtype: int64

In [432]:
df_short = df_short.dropna(subset=["kategori", "stasiun_clean"]).copy()

In [433]:
num_cols = [
    "pm_sepuluh",
    "pm_duakomalima",
    "sulfur_dioksida",
    "karbon_monoksida",
    "ozon",
    "nitrogen_dioksida",
    "delta_hari"
]


for col in num_cols:
    df_short[col] = pd.to_numeric(df_short[col], errors="coerce")

df_short[num_cols] = df_short[num_cols].fillna(0)

In [434]:
df_short.isna().sum()

periode_data                 0
tanggal                      0
stasiun                      0
pm_sepuluh                   0
pm_duakomalima               0
sulfur_dioksida              0
karbon_monoksida             0
ozon                         0
nitrogen_dioksida            0
max                          0
parameter_pencemar_kritis    0
kategori                     0
id                           0
stasiun_clean                0
tanggal_raw                  0
delta_hari                   0
dtype: int64

In [435]:
df_short.to_csv("short_CLEANED.csv", index=False)